<a href="https://colab.research.google.com/github/sayarghoshroy/Augment4Gains/blob/main/classification_test_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
%%capture
# Suppressing cell output
!pip install torch
import torch

In [3]:
cuda_available = torch.cuda.is_available()
print('CUDA is available: ' + str(cuda_available))
print('PyTorch version: ' + str(torch.__version__))
if cuda_available:
  torch.device('cuda')

CUDA is available: True
PyTorch version: 1.8.0+cu101


In [4]:
!nvidia-smi

Mon Mar 15 07:09:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os
import time
import sys
import json
import numpy as np
import pickle
import shutil

In [6]:
%%capture
# Suppressing cell output
!pip install protobuf
!pip install simpletransformers

# Note: If you're facing issues on Colab
# Restart and rerun from this cell

In [7]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report
import simpletransformers
import logging
import pandas as pd

In [8]:
# Set data name and path
data_name = 'reddit_no_augment'
data_path = 'drive/My Drive/Augment4Gains/data/reddit'
# For referencing data stored on Google drive like the above
# Mount drive first

# Expectation:
# data_path directory should contain train, val, test jsons
# data-points should be present as a list of dicts
# with each dict having a 'source', and a 'target'

with open(data_path + '/' + 'train.json', 'r+') as f:
  raw_train = json.load(f)

with open(data_path + '/' + 'val.json', 'r+') as f:
  raw_val = json.load(f)

with open(data_path + '/' + 'test.json', 'r+') as f:
  raw_test = json.load(f)

In [9]:
# Verifying loaded data
assert type(raw_train) == type(raw_val)
assert type(raw_train) == type(raw_test)
print('Raw data object type: ' + str(type(raw_train)))
print()

print('Fields in the raw data: ')
unit = raw_train[0]

for key in unit:
  print('• ' + str(key))

Raw data object type: <class 'list'>

Fields in the raw data: 
• set
• main_index
• index
• type
• source
• target


In [10]:
# To test out the procedure with small amounts of data
global_testing_mode = 1
global_testing_unit_count = 512

In [11]:
print('Number of Samples in: ')
print('• train: ' + str(len(raw_train)))
print('• val: ' + str(len(raw_val)))
print('• test: ' + str(len(raw_test)))

# Defining mappins for training
def create_set(set_name = 'train'):
  global raw_train, raw_val, raw_test
  global global_testing_mode, global_testing_unit_count
  work_on = None

  if set_name == 'train':
    work_on = raw_train
  elif set_name == 'val':
    work_on = raw_train
  elif set_name == 'test':
    work_on = raw_train
  else:
    print('Invalid Data Split.')
    return -1
  
  data_size = len(work_on)
  if global_testing_mode:
    data_size = global_testing_unit_count

  data = []
  for index in range(data_size):
    unit = [work_on[index]['source'], work_on[index]['target']]
    data.append(unit)

  return data

train = create_set('train')
val = create_set('val')
test = create_set('test')

Number of Samples in: 
• train: 15619
• val: 2231
• test: 4464


In [12]:
# Getting number of positive and negative samples in train split
total_in_train = len(train)
positive_in_train = 0

for unit in train:
  positive_in_train += unit[1]

print('Number of positive samples: ' + str(positive_in_train))
print('Number of negative samples: ' + str(total_in_train - positive_in_train))

# Weights to correct the class imbalance
greater_class_count = max((total_in_train - positive_in_train), positive_in_train)
class_weights = [greater_class_count / (total_in_train - positive_in_train),
                 greater_class_count / positive_in_train]

Number of positive samples: 115
Number of negative samples: 397


In [13]:
# Defining dataframes
train_df = pd.DataFrame(train)
train_df.columns = ['source', 'label']

val_df = pd.DataFrame(val)
val_df.columns =['source', 'label']

# Verifying correctness
train_df

,source,label
0,if you give me that salsa i will love you fore...,0
1,that rotten cunt . next thing you know shell s...,1
2,it was a lawful arrest you moron . you do n't ...,0
3,what a gem from aeons ago .,0
4,u had referred to me as a moron yet you also d...,0
...,...,...
507,> i think whoever payed attention in school pr...,0
508,[ removed ],0
509,so the legal argument here is whether 'private...,1
510,what a twat ! but a twat with a good rack .,1


In [14]:
%%capture
# Leveraging a pre-trained Transformer Model

model_index = 0
# Set 0 for bert-base, 1 for roberta-base

model_loc = ['bert-base-uncased', 'roberta-base'][model_index]
model_type = ['bert', 'roberta'][model_index]

is_lower = False
if model_index == 0:
  is_lower = True

length_setting = 256
model_name = model_loc + '_' + data_name + '_' + str(length_setting)
cache_name = model_name + '_cache_dir'

batch_size = 32
num_epochs = 8
num_gpus = 1

if global_testing_mode == 1:
  model_name += '_testing'
  num_epochs = 2
  length_setting = 64

model_args = ClassificationArgs(train_batch_size = batch_size,
                                max_seq_length = length_setting,
                                save_steps = -1,
                                n_gpu = num_gpus,
                                num_train_epochs = num_epochs,
                                evaluate_during_training = True,
                                overwrite_output_dir = True,
                                save_eval_checkpoints = False,
                                save_model_every_epoch = False,
                                cache_dir = cache_name,
                                fp16 = True,
                                manual_seed = 42,
                                do_lower_case = is_lower,
                                best_model_dir = model_name)

model = ClassificationModel(model_type,
                            model_loc,
                            use_cuda = cuda_available,
                            args = model_args,
                            num_labels = 2,
                            weight = class_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
# Training

start = time.time()
model.train_model(train_df, eval_df = val_df)
end = time.time()
time_to_train = int(round(end - start))

hours = int(time_to_train / 3600)
minutes = int(int(time_to_train % 3600) / 60)
seconds = int(time_to_train % 60)
print()
print('Number of Epochs: ' + str(num_epochs))
print('Maximum Sequence Length: ' + str(length_setting))
print('Batch size: ' + str(batch_size))
print('Time taken for training: ' + str(hours).zfill(2) + ':' + str(minutes).zfill(2) + ':' + str(seconds).zfill(2))

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/16 [00:00<?, ?it/s]


Number of Epochs: 2
Maximum Sequence Length: 64
Batch size: 32
Time taken for training: 00:00:53


In [16]:
# Inference
infer_now = True

if infer_now == True:
  model = ClassificationModel(model_type, model_name)
  print('Using Model: ' + str(model_name))
  print()
  
  val_sources = [unit[0] for unit in val]
  test_sources = [unit[0] for unit in test]

  val_targets = [unit[1] for unit in val]
  test_targets = [unit[1] for unit in test]

  # Evaluation on val data
  print('Results on the validation split: ')
  val_predictions, val_outputs = model.predict(val_sources)
  print(classification_report(val_targets, val_predictions))
  print()

  # Evaluation on test data
  print('Results on the test split: ')
  test_predictions, test_outputs = model.predict(test_sources)
  print(classification_report(test_targets, test_predictions))

Using Model: bert-base-uncased_reddit_no_augment_256_testing

Results on the validation split: 


  0%|          | 0/64 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.97      0.86      0.91       397
           1       0.65      0.92      0.77       115

    accuracy                           0.87       512
   macro avg       0.81      0.89      0.84       512
weighted avg       0.90      0.87      0.88       512


Results on the test split: 


  0%|          | 0/64 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.97      0.86      0.91       397
           1       0.65      0.92      0.77       115

    accuracy                           0.87       512
   macro avg       0.81      0.89      0.84       512
weighted avg       0.90      0.87      0.88       512



In [17]:
compress_model = True
if compress_model == True:
  shutil.make_archive(model_name, 'zip', model_name)
  shutil.make_archive(cache_name, 'zip', cache_name)

In [18]:
# ^_^ Thank You